Transform Orders Data - String to JSON
- 1.Pre_process the JSON string to fix the data quality issues
- 2 Transform JSON String to JSON object
- 3.Write tarnsformed data to the silver schema

In [0]:
df_orders = spark.table("gizmobox.bronze.orders_vw")

In [0]:
display(df_orders)

In [0]:
from pyspark.sql import functions as F

# Step 1: Fix the JSON string (Handle the space correctly!)
df_fixed_orders = df_orders.select(
    F.regexp_replace(
        "value", 
        # The \s? handles the space after the colon
        r'"order_date":\s?(\d{4}-\d{2}-\d{2})', 
        r'"order_date": "$1"'
    ).alias("fixed_value_cleaned")  # Naming it 'cleaned' to match your next step
)

# Step 2: Infer the schema from the correctly fixed column
df_with_Schema = df_fixed_orders.select(
    F.schema_of_json(F.col("fixed_value_cleaned")).alias("Schema")
)

display(df_with_Schema.limit(1))

In [0]:
orders_schema ='STRUCT<customer_id: BIGINT, items: ARRAY<STRUCT<category: STRING, details: STRUCT<brand: STRING, color: STRING>, item_id: BIGINT, name: STRING, price: BIGINT, quantity: BIGINT>>, order_date: STRING, order_id: BIGINT, order_status: STRING, payment_method: STRING, total_amount: BIGINT, transaction_timestamp: STRING>'


In [0]:
df_josn_orders = (
    df_fixed_orders.select(
        F.from_json(F.col("fixed_value_cleaned"), orders_schema).alias("orders")
    )
)
display(df_josn_orders
)

In [0]:
df_josn_orders.writeTo("gizmobox.silver.py_orders").createOrReplace()